## d2armor - Analyze all armor in your Destiny 2 vault 

A python jupyter notebook that can analyze all armor in your Destiny 2 vault and tell you what is great and what is worth sharding.

This notebook assumes that your vault information has already been downloaded into `data/profile.json` via the `d2profile.ipynb` jupyter notebook.

In [ ]:
# use the data/profile.json downloaded by `d2profile.ipynb` to load your profile containing all armor and weapons
import os
import json

os.makedirs("data", exist_ok=True)

# assert that the json files exist
assert os.path.exists(
    "data/profile.json"
), "Profile file not found. Run the d2profile.ipynb notebook first to generate it."
assert os.path.exists(
    "data/item_definitions.json"
), "Item definitions file not found. Run the d2profile.ipynb notebook first to generate it."
assert os.path.exists(
    "data/stat_definitions.json"
), "Stat definitions file not found. Run the d2profile.ipynb notebook first to generate it."

with open("data/profile.json", "r") as file:
    profile = json.load(file)

print("Character profile loaded at:", profile["responseMintedTimestamp"])

with open("data/item_definitions.json", "r") as file:
    item_definitions = json.load(file)

with open("data/stat_definitions.json", "r") as file:
    stat_definitions = json.load(file)

In [ ]:
# extract all armor pieces out of the profile.  It retrieves from the vault, character inventory, and character equipment
from src.armor import ProfileArmor

profile_armor = ProfileArmor(profile, item_definitions, stat_definitions)

unfiltered_armor_dict = profile_armor.get_armor_dict()
unfiltered_armor_dict

### Filter ignored armor

Ignore any armor in `data/ignored-armor.json` - if you have armor tagged as junk/infuse in DIM, you can export that as a CSV then identify the values to ignore
Expected format is a list of values that have an `instance_id` property, ex: 

```
[
    { "instance_id": 6917530015478829219},
    { "instance_id": 6917529815104854677}
]
```

Other fields can be on the object, we only care about filtering `instance_id` values here

In [ ]:
import os
import json

ignored_armor = []
if os.path.exists("data/ignored-armor.json"):
    with open("data/ignored-armor.json", "r") as file:
        ignored_armor = json.load(file)
        ignored_ids = [armor["instance_id"] for armor in ignored_armor]

    armor_dict = {}
    filtered_armor_dict = {}

    for armor in unfiltered_armor_dict.values():
        if armor.instance_id not in ignored_ids:
            armor_dict[armor.instance_id] = armor
        else:
            filtered_armor_dict[armor.instance_id] = armor

    print(
        f"data/ignored-armor.json contains {len(ignored_armor)} instance_id values. Filtered out {len(unfiltered_armor_dict) - len(armor_dict)} armor pieces."
    )
else:
    print("No ignored armor found")
    armor_dict = unfiltered_armor_dict

filtered_armor_dict

In [ ]:
import pandas as pd

# create a dataframe from the armor dictionary
armor_df = pd.DataFrame(
    [
        {
            **vars(armor),
            "total_stats": armor.total_stats,
            "is_exotic": armor.is_exotic,
            "class_slot": armor.class_slot,
        }
        for armor in armor_dict.values()
    ]
)

armor_df

In [ ]:
# create a graph that shows total stats by class and slot for all armor in the vault.  The larger the circle, the more we have with that stat total
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sort by class slot:
armor_df.sort_values("class_slot", ascending=True, inplace=True)

# Create a color map for the classes
color_map = {
    "Hunter": mcolors.CSS4_COLORS["skyblue"],
    "Titan": mcolors.CSS4_COLORS["lightcoral"],
    "Warlock": mcolors.CSS4_COLORS["khaki"],
}
armor_df["color"] = armor_df["d2_class"].map(color_map)

# Calculate the counts for each 'class_slot' value
counts = armor_df["class_slot"].value_counts()

plt.figure(figsize=(10, 6))
# Pass the counts as the 's' argument to plt.scatter()
plt.scatter(
    armor_df["class_slot"],
    armor_df["total_stats"],
    s=counts[armor_df["class_slot"]] * 2,
    color=armor_df["color"],
)
plt.xlabel("Slot + Class")
plt.ylabel("Total Stats")
plt.title("Total Stats by Slot and Class")
plt.xticks(rotation=90)
plt.show()

### Create ProfileOutfits which will let us calculate permutations of all outfits in our vault

In [ ]:
from src.armor import ProfileOutfits

# uncomment if we want to see only armor that isn't exotic
# non_exotic_armor_dict = {k: v for k, v in armor_dict.items() if not v.is_exotic}
# profile_outfits = ProfileOutfits(non_exotic_armor_dict)

# or uncomment this to use all armor in the vault
profile_outfits = ProfileOutfits(armor_dict)

### Identify all armor that is eclipsed by another piece of armor and can be safely deleted

In [ ]:
# print out all armor that has the same or worse stats than another piece of armor of the same rarity and type
for lesser_armor, greater_armor in profile_outfits.find_eclipsed_armor():
    print(f"id:{lesser_armor.instance_id} OR id:{greater_armor.instance_id}")
    print(
        f"mob {lesser_armor.mobility}\tres {lesser_armor.resilience}\trec {lesser_armor.recovery}\tdis {lesser_armor.discipline}\tint {lesser_armor.intellect}\tstr {lesser_armor.strength}\t name {lesser_armor.item_name} <-- can be deleted"
    )
    print(
        f"mob {greater_armor.mobility}\tres {greater_armor.resilience}\trec {greater_armor.recovery}\tdis {greater_armor.discipline}\tint {greater_armor.intellect}\tstr {greater_armor.strength}\t name {greater_armor.item_name} <-- is equal or better"
    )

### Generate all possible outfits for each class type

In [ ]:
# generate outfit permutations for each class
d2_class = "Hunter"
hunter_outfits = profile_outfits.generate_class_outfits(d2_class)
print(f"Generated {len(hunter_outfits)} outfit permutations for {d2_class}")

d2_class = "Titan"
titan_outfits = profile_outfits.generate_class_outfits(d2_class)
print(f"Generated {len(titan_outfits)} outfit permutations for {d2_class}")

d2_class = "Warlock"
warlock_outfits = profile_outfits.generate_class_outfits(d2_class)
print(f"Generated {len(warlock_outfits)} outfit permutations for {d2_class}")

## Pick a class to work with for the rest of the notebook

In [ ]:
# modify to the class you want to work with below
# d2_class, outfits = ("Hunter", hunter_outfits)
# d2_class, outfits = ("Titan", titan_outfits)
d2_class, outfits = ("Warlock", warlock_outfits)

In [ ]:
# given the outfit permutations above PinnacleOutfits will generate a dataframe with weighted stats for each outfit
# this lets us see which armor pieces are in outfits that are "pinnacle" (have the highest total stats for a given stat combination)
from src.armor import PinnacleOutfits

pinnacle_outfits = PinnacleOutfits(outfits)
weighted_outfits_df = pinnacle_outfits.weighted_outfits_df
weighted_outfits_df

In [ ]:
# finds the max value in each of the "weighted" columns grouped by num_artifice, then filter to those rows
weighted_outfits_max_df = pinnacle_outfits.weighted_outfits_max_df
weighted_outfits_max_df

In [ ]:
# filters the weighted outfits to only those outfits that have a weighted stat total equal to the maximum stat value for that exotic_hash
# this tells us which outfits hit that peak stat value so we can tell what armor pieces contribute
pinnacle_outfits_df = pinnacle_outfits.pinnacle_outfits_df
pinnacle_outfits_df

In [ ]:
# create a DIM query that will show armor for the given class that could be safely deleted
# this is defined as armor that isn't in any pinnacle outfit for the currently selected class

# iterate over the outfits in pinnacle_outfits_df and create a set of armor pieces that are in the outfits
pinnacle_armor = set()
for outfit in pinnacle_outfits_df.iter_rows():
    for i in range(6, 11):
        pinnacle_armor.add(outfit[i])

# emit the values in pinnacle_armor with `id:` in front of them so they can be used in the DIM search bar and joined together with an `OR`
dim_query = " OR ".join([f"id:{item}" for item in pinnacle_armor])

print(f"is:{d2_class} is:armor NOT ({dim_query})")

### Print out the exotic pieces that we can reach triple 100s with

it will emit the combinations of stats that we can reach triple 100s with for each piece of exotic armor

It is actually looking for 3 stats that together are 250 points in total.  This would allow the use of five 10-point armor mods to hit triple 100

There is no consideration for stat modifications that class fragments bring into the mix.  It assumes neutral stat modifications outside of armor.

In [ ]:
import itertools
from collections import defaultdict

# Initialize an empty dictionary
exotic_combinations = defaultdict(set)

# Define the attributes
attributes = ["mob", "res", "rec", "dis", "int", "str"]

stat_column_combinations = list(itertools.combinations(range(6), 3))

for outfit in pinnacle_outfits_df.iter_rows():
    exotic_hash = outfit[11]  # Get the exotic_hash for the outfit

    for perm in stat_column_combinations:
        # Calculate the sum of the attributes
        attr_sum = sum(outfit[attr] for attr in perm)

        # If the sum is >= 250, add the permutation to the set
        if attr_sum >= 250:
            exotic_combinations[exotic_hash].add(perm)

# create a dict of the armor_hash to the name of the armor piece
armor_hash_to_name = {armor.item_hash: armor.item_name for armor in armor_dict.values()}

# Print the exotic_combinations dictionary
if len(exotic_combinations.items()) == 0:
    print("Sorry, no triple-100s found, guess you'll have to play more")
else:
    for exotic_hash, combinations in exotic_combinations.items():
        if exotic_hash == ProfileOutfits.NO_EXOTIC_HASH:
            print("No Exotic")
        else:
            # iterate over the values in armor_dict to find the name of the exotic armor piece
            print(f"Exotic: {armor_hash_to_name[exotic_hash]} -- {exotic_hash}")

        # Sort the combinations alphabetically before printing
        for combination in sorted(
            combinations, key=lambda x: [attributes[i] for i in x]
        ):
            print([attributes[i] for i in combination])

### Print out all Legendary Armor Pieces and the exotic stat combinations where this armor piece was in a pinnacle outfit

It is sorted by the number of pinnacle outfits the armor piece was in, look at the bottom of the list for armor pieces that are only in pinnacle outfits that you don't care about

If this armor piece isn't the only one that can make this stat combo, it will print the stat combination out between `~` characters

If all stat combos are marked like `~res/dis/str~` that means that piece of armor can be safely replaces with another piece of armor and has no unique combinations.

```
Tusked Allegiance Hood -- Helmet -- id:6917529855693975489 -- m:2 r:20 r:10 d:23 i:2 s:8 -- total pinnacle outfits: 8 -- unique pinnacle outfits: 2
    No Exotic - 0 - ~res/dis/str~
    Briarbinds - 0 - ~res~
    Necrotic Grip - 0 - ~dis/str~
    Karnstein Armlets - 1 - res/dis
    Aeon Soul - 0 - ~dis/str~
    Phoenix Protocol - 1 - res/dis/str
    Wings of Sacred Dawn - 0 - ~dis/str~
    Rain of Fire - 0 - ~res/dis/str~
```
 
The Tusked Allegiance Hood was in 8 pinnacle outfits, but only 2 of them were unique to this armor piece
- `res/dis` on Karnstein Armlets
- `res/dis/str` on Phoenix Protocol

The other stat combinations were in other armor pieces, so they are marked as not unique


The output is in descending order, so the least valuable armor pieces are at the bottom

In [ ]:
from src import report
import importlib

importlib.reload(report)
report.legendary_armor_to_pinnacle_outfits_report(
    d2_class, armor_dict, pinnacle_outfits_df
)

In [ ]:
# prints out exotic armor pieces and the stat combinations where this armor piece was in a pinnacle outfit
# sorts by exotic name so you can compare the stat combinations for each exotic
importlib.reload(report)
report.exotic_armor_to_pinnacle_outfits_report(
    d2_class, armor_dict, pinnacle_outfits_df
)

In [ ]:
import polars as pl
from polars import col


# if you want to find all the outfits for a specific armor piece
def find_exotic_outfits_df(exotic_name, armor_dict, outfits_df):
    exotic_hash = ProfileOutfits.NO_EXOTIC_HASH
    for armor in armor_dict.values():
        if armor.item_name == exotic_name:
            exotic_hash = armor.item_hash
            break

    return outfits_df.filter(col("exotic_hash") == exotic_hash)


exotic_outfits_df = find_exotic_outfits_df(
    "Starfire Protocol", armor_dict, pinnacle_outfits_df
)

exotic_outfits_df

In [ ]:
# find a particular exotic armor piece and stat combo you're interested in
stats = ["resilience", "discipline", "strength"]
weighted_column_name = "weighted_" + "_".join(stats)
weighted_max_column_name = weighted_column_name + "_max"
filtered_exotic_outfits_df = exotic_outfits_df.filter(
    col(weighted_column_name) == col(weighted_max_column_name)
)
filtered_exotic_outfits_df

In [ ]:
import matplotlib.pyplot as plt

# Reshape the DataFrame to long format
long_df = pinnacle_outfits_df.melt(
    value_vars=[
        "mobility",
        "resilience",
        "recovery",
        "discipline",
        "intellect",
        "strength",
    ],
)

# Rename the columns to 'stat' and 'points'
long_df = long_df.with_columns(
    long_df["variable"].alias("stat"),
    long_df["value"].alias("points"),
)

# Count the number of outfits for each point total for each stat
counts_df = long_df.group_by(["stat", "points"]).agg(pl.count("stat").alias("count"))

# Convert to pandas for easier plotting
counts_df = counts_df.to_pandas()

# Create a scatter plot
plt.figure(figsize=(10, 6))
for stat in [
    "mobility",
    "resilience",
    "recovery",
    "discipline",
    "intellect",
    "strength",
]:
    stat_df = counts_df[counts_df["stat"] == stat]
    plt.scatter(
        stat_df["points"],
        stat_df["count"],  # Y-axis
        alpha=0.5,
        label=stat,
    )
plt.xlabel("Points")
plt.ylabel("Count")
plt.title("Number of Points by Stat")
plt.legend()
plt.show()

### Scratch for testing specific outfits and comparing against DIM

In [ ]:
def print_row_weighted_vs_max(row):
    # print the row so that we show the weighted value compared to the weighted max value
    row = row.to_dict()
    for key, value in row.items():
        if not (key.startswith("weighted_") or key.endswith("_max")):
            print(f"{key}: {value[0]}")
        if key.startswith("weighted_") and not key.endswith("_max"):
            max_key = key + "_max"
            real_value = value[0]
            max_value = row[max_key][0]
            if real_value == max_value:
                print(
                    f"{key}: {value[0]} == {row[max_key][0]} ************************************"
                )
            else:
                print(f"{key}: {value[0]} < {row[max_key][0]}")


def print_outfit_stats(row):
    helmet_id = row["helmet"][0]
    gauntlets_id = row["gauntlets"][0]
    chest_id = row["chest_armor"][0]
    leg_id = row["leg_armor"][0]
    class_item_id = row["class_item"][0]

    mobility = 0
    resilience = 0
    recovery = 0
    discipline = 0
    intellect = 0
    strength = 0

    for id in [helmet_id, gauntlets_id, chest_id, leg_id, class_item_id]:
        armor = armor_dict[id]
        mobility += armor.mobility
        resilience += armor.resilience
        recovery += armor.recovery
        discipline += armor.discipline
        intellect += armor.intellect
        strength += armor.strength
        print(
            f"{armor.mobility}\t{armor.resilience}\t{armor.recovery}\t{armor.discipline}\t{armor.intellect}\t{armor.strength}\t{armor.total_stats}\t{armor.is_artifice}\t{armor.item_name}\t{armor.instance_id}"
        )

    print(
        f"{mobility}\t{resilience}\t{recovery}\t{discipline}\t{intellect}\t{strength}\t<-- base outfit stats"
    )
    print(
        f"{row['mobility'][0]}\t{row['resilience'][0]}\t{row['recovery'][0]}\t{row['discipline'][0]}\t{row['intellect'][0]}\t{row['strength'][0]}\t<-- outfit with applied artifice + masterwork & rounded to useful tiers"
    )

In [ ]:
# find rows in ploutfits_df that are duplicate rows for all fields - should not happen
pinnacle_outfits_df.filter(pinnacle_outfits_df.is_duplicated())

In [ ]:
# one of my specific outfits I can see in DIM - shows all permutations of this outfit that have unique stat totals
helmet_id = 6917529862437575151
gauntlets_id = 6917529838031225999
chest_id = 6917529838898582109
leg_id = 6917530017559101392
class_item_id = 6917529583788947730

outfit = weighted_outfits_df.filter(
    (weighted_outfits_df["helmet"] == helmet_id)
    & (weighted_outfits_df["gauntlets"] == gauntlets_id)
    & (weighted_outfits_df["chest_armor"] == chest_id)
    # & (weighted_outfits_df["leg_armor"] == leg_id)
    & (weighted_outfits_df["class_item"] == class_item_id)
)

# for i in range(outfit.height):
#     row = outfit[i]
#     print_outfit_stats(row)
#     print_row_weighted_vs_max(row)
outfit